<a href="https://colab.research.google.com/github/Khik2219/Assignment4/blob/main/Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import math
import numpy as np
import pandas as pd
from tensorflow import data as tf_data
import matplotlib.pyplot as plt

from tensorflow import keras
from keras import layers
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.layers import Input, Dense, Add
from tensorflow.keras.models import Model
from sklearn.datasets import fetch_covtype
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from tensorflow.keras.utils import to_categorical

In [2]:
data = fetch_covtype()
X = data.data
y = data.target
print(f"Dataset shape: {X.shape}")

Dataset shape: (581012, 54)


In [3]:
print(X.shape)  # (581012, 54)
print(y.shape)  # (581012,)
print(np.unique(y))

(581012, 54)
(581012,)
[1 2 3 4 5 6 7]


In [5]:
#Train and test split
x_train, X_temp, Y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
x_test, x_val, Y_test, Y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

#Data Preprocessing
#Handling missing values using sklearn imputer
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(x_train)
X_test = imputer.transform(x_test)
X_val = imputer.transform(x_val)
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
X_val = X_val.astype("float32")

#Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.transform(X_val)

#Encoding categorical values
nb_classes = 7
Y_train = Y_train-1 #scale so first class is now 0th class and so on
Y_test = Y_test-1
Y_val = Y_val-1
y_train = to_categorical(Y_train, nb_classes)
y_test = to_categorical(Y_test, nb_classes)
y_val = to_categorical(Y_val, nb_classes)



In [6]:
print(X_train_scaled.shape)
print(X_test_scaled.shape)
print(X_val_scaled.shape)
print(y_train.shape)
print(y_test.shape)
print(y_val.shape)
print(np.asarray(range(nb_classes)))
print(y_train[0].astype(np.int32))

(348607, 54)
(116202, 54)
(116203, 54)
(348607, 7)
(116202, 7)
(116203, 7)
[0 1 2 3 4 5 6]
[0 1 0 0 0 0 0]


In [7]:
inputs = keras.Input(shape=(54,))
dense_layer1 = Dense(32, activation='relu')(inputs)
dense_layer2 = Dense(32, activation='relu')(dense_layer1)
residual_connection = Add()([dense_layer2, dense_layer1])
dense_layer3 = Dense(32, activation='relu')(residual_connection)
dense_layer4 = Dense(32, activation='relu')(dense_layer3)
projection = Dense(32)(inputs) #match input shape to layer shape
skip_connection = Add()([dense_layer4, projection])
dense_layer5 = Dense(32, activation='relu')(skip_connection)
outputs = Dense(7, activation='softmax')(dense_layer5)

model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
model.save('covtype_mlp_model.h5')

In [9]:
#Choosing single batch of 128
batch_size = 128
X_batch = X_train_scaled[:batch_size]
y_batch = y_train[:batch_size]

In [10]:
history = model.fit(X_batch, y_batch,epochs=200,batch_size=batch_size, verbose=2,validation_data=(X_val_scaled, y_val))

Epoch 1/200
1/1 - 11s - 11s/step - accuracy: 0.0547 - loss: 2.4867 - val_accuracy: 0.0822 - val_loss: 2.5103
Epoch 2/200
1/1 - 7s - 7s/step - accuracy: 0.0781 - loss: 2.3633 - val_accuracy: 0.1031 - val_loss: 2.4102
Epoch 3/200
1/1 - 3s - 3s/step - accuracy: 0.1094 - loss: 2.2508 - val_accuracy: 0.1316 - val_loss: 2.3180
Epoch 4/200
1/1 - 5s - 5s/step - accuracy: 0.1328 - loss: 2.1481 - val_accuracy: 0.1639 - val_loss: 2.2331
Epoch 5/200
1/1 - 5s - 5s/step - accuracy: 0.1719 - loss: 2.0547 - val_accuracy: 0.1960 - val_loss: 2.1548
Epoch 6/200
1/1 - 5s - 5s/step - accuracy: 0.2109 - loss: 1.9672 - val_accuracy: 0.2281 - val_loss: 2.0821
Epoch 7/200
1/1 - 5s - 5s/step - accuracy: 0.2500 - loss: 1.8850 - val_accuracy: 0.2612 - val_loss: 2.0143
Epoch 8/200
1/1 - 5s - 5s/step - accuracy: 0.3047 - loss: 1.8089 - val_accuracy: 0.2955 - val_loss: 1.9507
Epoch 9/200
1/1 - 3s - 3s/step - accuracy: 0.3438 - loss: 1.7370 - val_accuracy: 0.3303 - val_loss: 1.8908
Epoch 10/200
1/1 - 5s - 5s/step - a

In [12]:
train_loss, train_acc = model.evaluate(X_batch, y_batch, verbose=2)
val_loss, val_acc = model.evaluate(X_val_scaled, y_val, verbose = 2)
print(f"Number of parameters: {model.count_params()}")
print(f"Final training loss: {train_loss:.4f}")
print(f"Final validation loss: {val_loss:.4f}")

4/4 - 0s - 19ms/step - accuracy: 0.9922 - loss: 0.0427
3632/3632 - 7s - 2ms/step - accuracy: 0.5928 - loss: 2.8353
Number of parameters: 7975
Final training loss: 0.0427
Final validation loss: 2.8353
